In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.spatial import Voronoi,voronoi_plot_2d
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import geopandas as gpd
import folium
import mplleaflet
import cartopy.crs as ccrs
from datetime import datetime

In [2]:
df=pd.read_csv("delhi_data.csv")

In [3]:
# Convert 'Time' column to datetime format if not already in that format
df['Time'] = pd.to_datetime(df['Time'])

# Define the time frame in AM/PM format
start_time = '06:00:00 AM'
end_time = '06:00:00 PM'

# Convert the time strings to datetime.time objects
start_time = datetime.strptime(start_time, '%I:%M:%S %p').time()
end_time = datetime.strptime(end_time, '%I:%M:%S %p').time()

# Filter data within the specified time frame
df = df[
    (start_time <= df['Time'].dt.time) & (df['Time'].dt.time <= end_time)
]


C:\Users\ayush\AppData\Local\Temp\ipykernel_22556\3627404429.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


In [4]:
df=df.drop("Time",axis=1)

In [5]:
df

,Latitude,Longitude,Number
1,28.552524,77.259932,2
4,28.604801,77.127429,3
5,28.603861,77.253639,7
6,28.515594,77.188770,3
7,28.632524,77.157224,1
...,...,...,...
193,28.614145,77.176147,1
194,28.569430,77.217060,5
195,28.618904,77.140064,8
197,28.683632,77.260030,9


In [6]:

# Function to duplicate rows based on the "Number" column
def duplicate_rows(row):
    return pd.concat([row] *int( row['Number']), axis=1).transpose()

# Apply the function to each row and concatenate the results
df = pd.concat(df.apply(duplicate_rows, axis=1).tolist(), ignore_index=True)


In [7]:
df.drop("Number",axis=1)

,Latitude,Longitude
0,28.552524,77.259932
1,28.552524,77.259932
2,28.604801,77.127429
3,28.604801,77.127429
4,28.604801,77.127429
...,...,...
592,28.681902,77.219352
593,28.681902,77.219352
594,28.681902,77.219352
595,28.681902,77.219352


In [8]:
df=df.values
df = df[:, [1, 0]]

In [9]:
dbscan=DBSCAN(eps=0.001, min_samples=5)
dbscan.fit(df)

DBSCAN(eps=0.001)

In [10]:

# Get cluster labels
labels = dbscan.labels_

# Separate core, boundary, and noise points
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True

boundary_points_mask = labels == -1  # Points on the boundary have label -1
noise_points_mask = labels == -1

core_points = df[core_samples_mask]
boundary_points = df[boundary_points_mask]
noise_points = df[noise_points_mask]

In [11]:
vor = Voronoi(df)
core_regions = []
boundary_regions = []
noise_regions = []

for i, region in enumerate(vor.regions):
    label = dbscan.labels_[i]
    if label != -1: 
        if label in dbscan.core_sample_indices_:
            core_regions.append(region)
        else:
            boundary_regions.append(region)
    else:
        noise_regions.append(region)


In [12]:
points=df
m = folium.Map(location=[28.6139, 77.2090], zoom_start=12)

for region in core_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='red', fill_opacity=0.4).add_to(m)

for region in boundary_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='yellow', fill_opacity=0.4).add_to(m)

for region in noise_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='green', fill_opacity=0.4).add_to(m)



# Save the map as an HTML file
m.save('night.html')

print("Voronoi plot saved as voronoi_map.html")

Voronoi plot saved as voronoi_map.html


# Night time

In [13]:
df=pd.read_csv("delhi_data.csv")

In [14]:
# Convert 'Time' column to datetime format if not already in that format
df['Time'] = pd.to_datetime(df['Time'])

# Define the time frame in AM/PM format
start_time = '06:00:00 AM'
end_time = '06:00:00 PM'

# Convert the time strings to datetime.time objects
start_time = datetime.strptime(start_time, '%I:%M:%S %p').time()
end_time = datetime.strptime(end_time, '%I:%M:%S %p').time()

# Filter data within the specified time frame
df = df[
    (start_time > df['Time'].dt.time) & (df['Time'].dt.time< end_time)
]


C:\Users\ayush\AppData\Local\Temp\ipykernel_22556\4050249200.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time'] = pd.to_datetime(df['Time'])


In [15]:
df=df.drop("Time",axis=1)

In [16]:

# Function to duplicate rows based on the "Number" column
def duplicate_rows(row):
    return pd.concat([row] *int( row['Number']), axis=1).transpose()

# Apply the function to each row and concatenate the results
df = pd.concat(df.apply(duplicate_rows, axis=1).tolist(), ignore_index=True)


In [17]:
df.drop("Number",axis=1)


,Latitude,Longitude
0,28.704914,77.123112
1,28.704914,77.123112
2,28.564348,77.199010
3,28.564348,77.199010
4,28.564348,77.199010
...,...,...
253,28.528233,77.144014
254,28.528233,77.144014
255,28.528233,77.144014
256,28.528233,77.144014


In [18]:
df=df.values
df = df[:, [1, 0]]

In [19]:
dbscan=DBSCAN(eps=0.001, min_samples=8)
dbscan.fit(df)

DBSCAN(eps=0.001, min_samples=8)

In [20]:

# Get cluster labels
labels = dbscan.labels_

# Separate core, boundary, and noise points
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[dbscan.core_sample_indices_] = True

boundary_points_mask = labels == -1  # Points on the boundary have label -1
noise_points_mask = labels == -1

core_points = df[core_samples_mask]
boundary_points = df[boundary_points_mask]
noise_points = df[noise_points_mask]

In [21]:
vor = Voronoi(df)
core_regions = []
boundary_regions = []
noise_regions = []

for i, region in enumerate(vor.regions):
    label = dbscan.labels_[i]
    if label != -1: 
        if label in dbscan.core_sample_indices_:
            core_regions.append(region)
        else:
            boundary_regions.append(region)
    else:
        noise_regions.append(region)


In [22]:
points=df
m = folium.Map(location=[28.6139, 77.2090], zoom_start=12)

for region in core_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='red', fill_opacity=0.4).add_to(m)

for region in boundary_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='yellow', fill_opacity=0.4).add_to(m)

for region in noise_regions:
    if -1 not in region and len(region) > 0:
        polygon = [vor.vertices[i] for i in region]
        polygon = np.array(polygon)
        polygon = [point[::-1] for point in polygon]
        polygon=Polygon(polygon)
        area=polygon.area
        if(area<0.002):
            polygon=list(polygon.exterior.coords)
            folium.Polygon(locations=polygon, color=False, fill=True, fill_color='green', fill_opacity=0.4).add_to(m)



# Save the map as an HTML file
m.save('day.html')

print("Voronoi plot saved as voronoi_map.html")

Voronoi plot saved as voronoi_map.html
